## PRACTICA OBTENCIÓN DE DATOS

### Web scraper de la página: http://gestiona.madrid.org/azul_internet/html/web/DatosEstacion24Accion.icm?ESTADO_MENU=2_1

### Libraries

In [6]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

### Functions

In [11]:
def get_options(url: str):
    """
    Función para extraer las estaciones del selector de la web.
    :url: url de la web con las estaciones disponibles.
    :return: dataframe con las estaciones y su código.
    """
    # Primero importamos el html de la página web
    response = requests.get(url)
    check_status(response.status_code)

    # Ahora lo guardamos con un objeto "soup"
    website_soup = bs(response.content)
    options = website_soup.find("select").find_all('option')
    options = [opt.text.strip() for opt in options if opt.text != '']
    df_options = pd.DataFrame({"Código": [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13 ,14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 110],
                               'Estaciones': options})
    return df_options

def get_html(url: str, n_station: int):
    """
    Función para extraer un objeto soup con el HTML de la url introducida.
    :url: string con la url de la web de madrid.org.
    :return: devuelve el objeto soup.
    """
    # Primero importamos el html de la página web
    response = request_station(url, n_station)
    check_status(response.status_code)
    
    # Ahora lo guardamos con un objeto "soup"
    website_soup = bs(response.content)
    display_station(website_soup)
    return website_soup

def request_station(url: str, n_station: int):
    """
    Función para realizar una petición POST con los datos del formulario introducidos.
    :url: dirección a la que se realiza la petición.
    :n_station: código de la estación de la que se desean recoger los datos.
    """
    payload = f'estaciones={n_station}&aceptar=Aceptar'
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    response = requests.post(url, headers=headers ,data=payload)
    return response

def display_station(soup: str):
    title = soup.find('h3', {'class': 'titGeo07gr1'}).find('strong').text
    estacion = title.split()[5]
    print(f"[display_station] Se han recogido los datos de la estación de {estacion}")

def get_tables(soup: str):
    """
    Función master para extraer las tablas del HTML.
    :soup: string con el objeto soup HTML.
    :return: devuelve una tupla con los dos dataframes extraídos.
    """
    # Vamos a probar con un find all
    tables = soup.find_all("table")
    df_tables = ()
    # Seleccionamos las tablas 2 y 3 de la página a escrapear
    for table in tables[2:4]:
        df = table_to_pandas(table)
        df_tables += (df,)
    return df_tables
        
def table_to_pandas(table_soup: str):
    """
    Función auxiliar para parsear una tabla HTML y convertirla a Dataframe.
    :table_soup: string con el html de la tabla.
    :return: dataframe con la tabla.
    """
    # Vamos a coger los nombres de las columnas de la tabla
    head = table_soup.find("thead").find_all("strong")[1:]
    columns = [column.text for column in head]
    columns.insert(0, 'HOURS')
    
    # Vamos a coger las filas de la tabla
    rows = []
    body = table_soup.find("tbody").find_all("tr")
    for row in body:
        values = row.find_all("td")
        values = [value.text.strip() for value in values]
        rows.append(values)
    
    # Construimos el dataframe
    df = pd.DataFrame(rows, columns = columns)
    return df

def check_status(code: int):
    """
    Comprueba que la url introducida es alcanzable por la petición.
    :code: código de respuesta tras la petición.
    """
    if code == 200 or code == 201:
        print(f"[check_status] Respuesta recibida correctamente con código {code}\n")
    else:
        raise("[check_status] Error al conectar con la url de la petición")

### CODE

In [12]:
# CODE
# Url con las posibles opciones
url_options = "http://gestiona.madrid.org/azul_internet/html/web/DatosRed24Accion.icm?ESTADO_MENU=2_1"

# Extraer la lista de opciones a elegir
get_options(url_options)

[check_status] Respuesta recibida correctamente con código 200



,Código,Estaciones
0,1,Getafe
1,2,Leganés
2,3,Alcalá de Henares
3,4,Alcobendas
4,5,Fuenlabrada
5,6,Móstoles
6,7,Torrejón de Ardoz
7,8,Alcorcón
8,9,Coslada
9,11,Colmenar Viejo


In [13]:
# Url con los datos a scrapear
url_estacion = "http://gestiona.madrid.org/azul_internet/html/web/DatosEstacion24Accion.icm?ESTADO_MENU=2_1"
codigo_estacion = 18

# Extracción de tablas del HTML
html = get_html(url_estacion, n_station=codigo_estacion)
pollution, weather = get_tables(html)

# Mostrar datos extraídos
display(pollution.head())
display(weather.head())

[check_status] Respuesta recibida correctamente con código 200

[display_station] Se han recogido los datos de la estación de Rivas


,HOURS,TIN,NO,NO2,PM10,NOX,O3
0,17:00,17.7,2,13,3,16,48
1,18:00,17.8,3,22,4,27,38
2,19:00,17.9,7,39,6,51,26
3,20:00,18.2,6,34,8,43,26
4,21:00,17.9,4,25,9,32,35


,HOURS,VV,DV,TMP,HR,PRE,RS,LL
0,17:00,0.9,140,11.7,94,939,17,0.00
1,18:00,0.6,174,11.7,93,940,1,1.00
2,19:00,1.2,260,10.7,94,940,1,0.60
3,20:00,1.2,137,10.6,95,940,1,0.00
4,21:00,1.4,107,10.4,95,941,1,0.00
